<a href="https://colab.research.google.com/github/JaquesZanon/JOB/blob/main/PREVISAO_DEMANDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWgnYvJd6q0YFJ2yq8uXM7CtvJ2IXMITQJBoj8jEe7CnEi3A2IG1HxY
Mounted at /content/drive


In [1]:
# Importa aquivo de texto (.txt)
import pandas as pd
df_original = pd.read_csv('/content/drive/MyDrive/JOB-Martins/DadosPrevisaoDemanda.csv',sep=';', decimal=',', engine='python')

df_original['PrimeiraDataSemana'] = pd.to_datetime(df_original['PrimeiraDataSemana'])
df_original['UltimaDataSemana'] = pd.to_datetime(df_original['UltimaDataSemana'])
df_original = df_original.sort_values(by=['PrimeiraDataSemana'])
df_original.head()


,SiglaUF,DsCategoria,DsSubCategoria,CdGrupoProdutoSimilar,DsGrupoProdutoSimilar,DsMarca,DsDivisaoFornecedor,ANO,PrimeiraDataSemana,UltimaDataSemana,SemanaAno,QtdeVendida,VlTotalBruto,VlTotalLiquido,VlReceitaLiquida,Semanas
0,BA,ACESSORIO ELETRICO,CAIXA DE LUZ,4200765,CXA.LUZ EMB.KRONA 4X2 AM.24X1,Krona,KRONA TUBOS E CONEXOES LTDA.,2020,2020-01-05,2020-01-05,18,1,23.25,23.25,21.10,1
850,BA,SABONETE,SABONETE EM BARRA,205138,SAB.GRANADO 12X90G GLIC.BENJO.,Granado,PONTELAND DISTRIBUICAO SA,2020,2020-01-05,2020-01-05,18,3,129.06,124.92,111.72,1
849,BA,SABONETE,SABONETE EM BARRA,205137,SAB.GRANADO 12X90G GLIC.AMEND.,Granado,PONTELAND DISTRIBUICAO SA,2020,2020-01-05,2020-01-05,18,3,144.30,139.68,124.92,1
848,BA,SABONETE,SABONETE EM BARRA,203130,SAB.GRANADO 12X90G GLIC.MEL,Granado,PONTELAND DISTRIBUICAO SA,2020,2020-01-05,2020-01-05,18,3,144.30,139.68,124.92,1
847,BA,SABONETE,SABONETE EM BARRA,201588,SAB.GRANADO 12X90G GLIC.CALE.,Granado,PONTELAND DISTRIBUICAO SA,2020,2020-01-05,2020-02-05,18,4,164.79,154.57,138.24,1


In [2]:
#dat_aux = pd.DataFrame(df_reduzido.groupby(['DsDivisaoFornecedor'])['CdGrupoProdutoSimilar'].unique())
dat_aux = pd.DataFrame(df_original.groupby(['DsDivisaoFornecedor'])['CdGrupoProdutoSimilar'].unique())
dat_aux

,CdGrupoProdutoSimilar
DsDivisaoFornecedor,
3M DO BRASIL LTDA,"[805, 97145, 803, 88, 818, 97147, 406632, 9714..."
3M DO BRASIL LTDA - DIV. FITAS,"[406207, 400433, 229, 406675, 406674, 400434, ..."
3M DO BRASIL LTDA - EPI,"[9000524, 9000527, 9000531]"
3M DO BRASIL LTDA - LIMPEZA,"[1700717, 1704060, 1702339, 200755, 203136, 20..."
3M DO BRASIL LTDA LIMPEZA &&MP,"[1702629, 1701347, 1701348, 1701261, 1701262, ..."
...,...
WHIRLPOOL S.A - L.BCA,"[636, 1041, 191, 2217224, 1793, 1794, 2213230,..."
WORLD COM IND MED VET COSM LIM,"[701869, 701761, 701768, 701767, 701762, 70176..."
WOW INDUSTRIA E COMERCIO LTDA,"[1600715, 1600716, 1600609, 1600455, 1600390, ..."


In [ ]:
# Precisamos instalar algumas Bibliotecas
!pip install utils
!pip install pmdarima # para autoARIMA
!pip install pyEDM # Para Empirical Dynamic Modeling
!pip install croston
!pip install gluonts # PAra DeepAR
!pip install mxnet # PAra Deep AR
!pip install prophet
!pip install tqdm
!pip install sklearn
!pip install --upgrade mxnet~=1.7 gluonts
!pip install darts
!pip install 'u8darts[all]'
!pip install skedm

In [3]:
def smape(a, f):
    return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

In [7]:
#def percentage_error(actual, predicted):
#   res = np.empty(actual.shape)
#    for j in range(actual.shape[0]):
#        if actual[j] != 0:
#            res[j] = (actual[j] - predicted[j]) / actual[j]
#        else:
#            res[j] = predicted[j] / np.mean(actual)
#    return res
#
#def mean_absolute_percentage_error(y_true, y_pred): 
#    return np.mean(np.abs(percentage_error(np.asarray(y_true), np.asarray(y_pred)))) * 100

In [4]:
from scipy import stats
import numpy as np
import pmdarima as pm
#from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
from croston import croston
from prophet import Prophet
from tqdm import tqdm
import time
import pyEDM
from darts import TimeSeries
from darts.metrics import mape
from darts.models import (
    NaiveSeasonal,
    NaiveDrift,
    ExponentialSmoothing,
    ARIMA,
    AutoARIMA,
    RegressionEnsembleModel,
    RegressionModel,
    Theta,
    FFT
)
import skedm as edm

In [ ]:
par={}
erro_arima={}
par_croston={}
erro_croston={}
erro_prophet={}
erro_EDM = {}
erro_naive = {}
erro_ExponentialSmoothing = {}
erro_Theta = {}
erro_FFT = {}

pred_arima ={}
pred_croston = {}
pred_prophet = {}
pred_EDM = {}
pred_Naive ={}
pred_ExponentialSmoothing={}
pred_Theta = {}
Pred_FFT = {}


#for j, i in zip(tqdm (range(0, 7),desc="Processando Fornecedor", ascii=False, ncols=75), tqdm (range(0,len(dat_aux.iloc[j,0])),desc="Processando Id Oriduto", ascii=False, ncols=75)):

for j in tqdm (range(0, 441),desc="Processando Fornecedor", ascii=False, ncols=75):
  
    try:
   # Selecionando os dados para ajustar os modelos
         filter1 = df_original["DsDivisaoFornecedor"]==dat_aux.index[j]
         #filter1 = df_original["DsDivisaoFornecedor"]=='SAINT GOBAIN DO BRASIL PRODUTOS IND'
         df_0 = df_original[filter1]
         df = df_0.groupby(['Semanas'])[['VlTotalBruto']].sum()
         we = pd.DataFrame(list(range(1,65,1)))
         we.columns=['Semanas']
         we.index=we.Semanas
         df = pd.concat([df, we], axis=1)
         df=df['VlTotalBruto']
         df.index = pd.DataFrame(pd.date_range('2020-01-05', freq='7D', periods=64), columns=['date']).iloc[:,0]
         df = pd.DataFrame(df).fillna(0)

      # Substituindo outliers
         z = np.abs(stats.zscore(df.VlTotalBruto))
         median=df.VlTotalBruto.median()
         df["VlTotalBruto"] = np.where(z>3, median, df["VlTotalBruto"])

      # Sets de treino e teste
         train_len = int(df.shape[0] * 1)
         train_data, test_data = df[:train_len], df[train_len:]
         test_data = train_data

      # Ajustando AutoArima
         stepwise_fit = pm.auto_arima(train_data, start_p=1, start_q=1, max_p=3, max_q=3, m=12,
                             start_P=0, seasonal=True, d=1, D=1, trace=False,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=True)  # set to stepwise
   
      # Armazenando resultados AutoArima
         sentence = [str(dat_aux.index[j])]
         sentence = '-'.join(sentence)
         aux = stepwise_fit.get_params()
         par[sentence]=aux
         forecasts = stepwise_fit.predict(test_data.shape[0])
         #mmae = mean_absolute_percentage_error(list(list(test_data.VlTotalBruto)), list(forecasts))
         mmae = smape(test_data.VlTotalBruto, forecasts)
         erro_arima[sentence]=mmae
         
  
      # Ajustando modelo Croston
         croston_pred = croston.fit_croston(df+0.000001,test_data.shape[0],'original')

      # Armazenando resultados Croston
         par_croston[sentence] = croston_pred
         #erro_croston[sentence] = mean_absolute_percentage_error(list(test_data.VlTotalBruto), croston_pred['croston_forecast'])
         erro_croston[sentence] = smape(test_data.VlTotalBruto, pd.DataFrame(croston_pred['croston_forecast']).iloc[:,0])

      # O modelo profet só funciona com duas colunas
         y = df.reset_index(drop=False)
         y.columns = ['ds', 'y']
         #y['floor'] = 0
         #y['cap'] = 200000
         train = y.iloc[:train_data.shape[0],:]
         test = y.iloc[train_data.shape[0]:,:]
         test=train

      #Ajustando modelo Prophet
         m = Prophet(weekly_seasonality = True,
                     yearly_seasonality = False,
                     daily_seasonality = False)
         m.fit(train,verbose=0)
         future = m.make_future_dataframe(periods=len(test))
         future['ds'] = pd.to_datetime(future['ds']).dt.date
         forecast = m.predict(future)
         one=test['y']
         twi=list(forecast.loc[:test_data.shape[0]-1,'yhat'])
         #erro_prophet[sentence] = mean_absolute_percentage_error(list(one),list(twi)) Assimetrico
         erro_prophet[sentence] = smape(one, twi) # simmetrico
         

         # O EDM agora, preparando o imput
         y = y[["ds","y"]]
         y.columns = ["Time", "Revenue"]
         y.Time = list(range(1,df.shape[0]+1,1))
         aaa=pyEDM.EmbedDimension(dataFrame = y, lib="1 64", pred="1 64", columns="Revenue",target = "Revenue",showPlot=False)
         maxE = aaa.rho.max()
         selectedE=aaa[aaa.rho==maxE]['E']
         X = y.Revenue+0.000001
         E = edm.Embed(X) #initiate the class
         max_lag = 20
         mi = E.mutual_information(max_lag)
         mi = pd.DataFrame(mi)
         mi['lag'] = range(0,20,1)
         mi.columns = ["mi", "lag"]
         maxMI = mi.mi.max()
         selectedMI=mi[mi.mi==maxMI]['lag']
         lag = int(selectedMI)
         embed = int(selectedE)
         predict = 20 #predicting out to double to lag
         X,y = E.embed_vectors_1d(1 if lag==0 else lag,3 if embed>5 else embed,predict)

         #split it into training and testing sets
         train_len = int(1*len(X))
         Xtrain = X[0:train_len]
         ytrain = y[0:train_len]
         Xtest = X[train_len:]
         ytest = y[train_len:]
         Xtest = Xtrain
         ytest = ytrain

         
         weights = 'distance' #use a distance weighting for the near neighbors
         M = edm.Regression(weights) # initiate the nonlinear forecasting class
         M.fit(Xtrain, ytrain) #fit the data (rebuilding the attractor)

         nn_list = [20]
         ypred = M.predict(Xtest,nn_list)
         ypred = pd.DataFrame(ypred[0])
         ypred.mean(axis=0)
         ypred = ypred.mean(axis=0)
         #erro_EDM[sentence] = mean_absolute_percentage_error(list(df.VlTotalBruto[:40]),list(ypred))
         erro_EDM[sentence] = smape(df.VlTotalBruto[:20],ypred)

         # PAra Naive, Prophet, exponential smothing, theta, FFT
         series = TimeSeries.from_dataframe(df.reset_index(), 'date', 'VlTotalBruto')
         train2, val = series.split_after(0.7)
         trains2=val=series
         model1 = NaiveDrift()
         model1.fit(train2+0.00000001)
         
         pred_val1 = model1.predict(len(val))
         erro_naive[sentence] = str(smape(series.values(),pred_val1.values()))

         model2 = ExponentialSmoothing()
         model2.fit(train2+0.00000001)

         pred_val2 = model2.predict(len(val))
         erro_ExponentialSmoothing[sentence] = str(smape(series.values(),pred_val2.values()))

         model3 = Theta()
         model3.fit(train2+0.00000001)

         pred_val3 = model3.predict(len(val))
         erro_Theta[sentence] = str(smape(series.values(),pred_val3.values()))

         model4 = FFT(trend='poly',trend_poly_degree = 1)
         model4.fit(train2+0.00000001)

         pred_val4 = model4.predict(len(val))
         erro_FFT[sentence] = str(smape(series.values(),pred_val4.values()))

    except:
      pass  

resultado_final = pd.concat([pd.DataFrame.from_dict(erro_arima,orient='index'),
           pd.DataFrame.from_dict(erro_croston,orient='index'),
           pd.DataFrame.from_dict(erro_prophet,orient='index'),
           pd.DataFrame.from_dict(erro_EDM,orient='index'),
           pd.DataFrame.from_dict(erro_naive,orient='index'),
           pd.DataFrame.from_dict(erro_ExponentialSmoothing,orient='index'),
           pd.DataFrame.from_dict(erro_Theta,orient='index'),
           pd.DataFrame.from_dict(erro_FFT,orient='index')   
           ], axis=1)
resultado_final.columns=["arima","croston", "prophet","EDM","naive","ExpSmooth","theta","fft"]
#resultado_final.to_csv('/content/drive/MyDrive/JOB-Martins/rfesultado_final.csv')
#pd.DataFrame.from_dict(erro_prophet,orient='index').to_csv('/content/drive/MyDrive/JOB-Martins/prophet.csv')
#pd.DataFrame.from_dict(erro_croston,orient='index').to_csv('/content/drive/MyDrive/JOB-Martins/croston.csv')
#pd.DataFrame.from_dict(erro,orient='index').to_csv('/content/drive/MyDrive/JOB-Martins/arima.csv')
#pd.DataFrame.from_dict(erro_EDM,orient='index').to_csv('/content/drive/MyDrive/JOB-Martins/EDM.csv')
print('Cmplete')

In [46]:
smape(df.VlTotalBruto[:40,],pd.DataFrame(ypred).iloc[:,0])

0.0

In [43]:
pd.DataFrame(ypred)

,0
0,4317.397504
1,4550.882913
2,4502.693751
3,4562.722922
4,4690.619997
5,4665.350835
6,4677.538751
7,4929.776248
8,5098.862920
9,5044.627504


In [52]:
resultado_final.to_csv('/content/drive/MyDrive/JOB-Martins/rfesultado_final.csv')

In [ ]:
resultado_final.to_excel('/content/drive/MyDrive/JOB-Martins/rfesultado_final.xlsx')

In [ ]:
df_original.columns

Index(['SiglaUF', 'DsCategoria', 'DsSubCategoria', 'CdGrupoProdutoSimilar',
       'DsGrupoProdutoSimilar', 'DsMarca', 'DsDivisaoFornecedor', 'ANO',
       'PrimeiraDataSemana', 'UltimaDataSemana', 'SemanaAno', 'QtdeVendida',
       'VlTotalBruto', 'VlTotalLiquido', 'VlReceitaLiquida', 'Semanas'],
      dtype='object')

In [ ]:
resultado = {}
df_2 = df
for j in range(0, 441):
   filter1 = df_original["DsDivisaoFornecedor"]==dat_aux.index[440]
   df_0 = df_original[filter1]
   df = df_0.groupby(['Semanas'])[['QtdeVendida']].sum()
   we = pd.DataFrame(list(range(1,65,1)))
   we.columns=['Semanas']
   we.index=we.Semanas
   df = pd.concat([df, we], axis=1)
   df=df['QtdeVendida']
   df.index = pd.DataFrame(pd.date_range('2020-01-05', freq='7D', periods=64), columns=['date']).iloc[:,0]
   df = pd.DataFrame(df).fillna(0)
   df.columns = [dat_aux.index[j]]
   df_2[dat_aux.index[j]]=df
   #resultado[dat_aux.index[j]] = df.reset_index()


In [ ]:
df

,QtdeVendida
Semanas,
1,246
2,750
3,1300
4,872
5,974
...,...
60,2962
61,1388
62,1383


In [ ]:

df_2.to_excel('/content/drive/MyDrive/JOB-Martins/quantidade_vendida_Series.xlsx')